<a href="https://colab.research.google.com/github/TheSkyAboveTheSky/Projet-PFA/blob/main/PFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [14]:
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import csv
import matplotlib.pyplot as plt
import numpy as np

# **Data**

In [17]:
def removeImages(data_dir) :
  # List all the images in the folder
  file_list = os.listdir(data_dir)
  # Remove them all
  for file_name in file_list:
      file_path = os.path.join(data_dir, file_name)
      if os.path.isfile(file_path):
          os.remove(file_path)
# Data Directory
data_dir = "data"
# Sort the list of our png files 
image_files = sorted(
    [file for file in os.listdir(data_dir) if file.endswith(".png")],
    key=lambda x: int(''.join(filter(str.isdigit, os.path.splitext(os.path.basename(x))[0])))
)
# List to store our images 
images = []
for image_file in image_files:
  image_path = os.path.join(data_dir, image_file)
  image = cv2.imread(image_path)
  images.append(image)
# Read labels from csv file
labels = []
with open('label.csv','r') as f:
  reader = csv.reader(f)
  # skip the columns name
  next(reader)
  for row in reader:
    labels.extend([row])
# Get the list of all unique characters in the labels
characters = sorted(list(set(char for arr in labels for string in arr for char in string)))
print("Number of images found:", len(images))
print("Number of labels found:", len(labels))
print("Number of unique characters:", len(characters))
print("Characters present:", characters)


Number of images found: 2
Number of labels found: 115
Number of unique characters: 64
Characters present: [' ', '#', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# **Define Our Variable**

In [22]:
img_width = 600
img_height = 200
max_length = 50
batch_size = 1
input_shape = (img_height,img_width,3)
num_classes = len(characters) +1

# **TRansform characters into numbers and numbers into characters**

In [21]:
char_to_num = keras.layers.StringLookup(
    vocabulary=characters, mask_token=None
)
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)